In [4]:
import datasets
import pandas as pd
from api_keys import GEMINI_API_KEY
from google import genai
from google.genai import types

/home/dilochan/Documents/projects/terraguard/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LLM API call

In [9]:
gemini_client = genai.Client(api_key=GEMINI_API_KEY)

Loading dataset containing prompts and sample Terraform code

In [5]:
from datasets import load_dataset

dataset = load_dataset("autoiac-project/iac-eval", split="test")

In [6]:
df = dataset.to_pandas()
print(df.columns)
print(df.shape)

Index(['Resource', 'Prompt', 'Rego intent', 'Difficulty', 'Reference output',
       'Intent'],
      dtype='object')
(458, 6)


In [8]:
df['Reference output']

0      terraform {\n  required_providers {\n    aws =...
1      provider "aws" {\n    region = "us-east-1"\n}\...
2      provider "aws" {\n  region = "us-east-1"\n}\n\...
3      terraform {\n  required_providers {\n    aws =...
4      terraform {\n  required_providers {\n    aws =...
                             ...                        
453    terraform {\n  required_providers {\n    aws =...
454    terraform {\n  required_providers {\n    aws =...
455    provider "aws" {\n  region = "us-west-1"\n}\n\...
456    terraform {\n  required_providers {\n    aws =...
457    terraform {\n  required_providers {\n    aws =...
Name: Reference output, Length: 458, dtype: object

### Prompts setup

In [2]:
import os

with open("templates/system_prompt.txt", "r") as f:
    system_prompt = f.read()

with open("templates/cot.txt", "r") as f:
    cot_prompt = f.read()

with open("templates/few-shot.txt", "r") as f:
    few_shot_prompt = f.read()
        

System Prompt

In [3]:
system_prompt

'You are TerraGuard, an AI agent that builds Cloud Infrastructure written in Terraform HCL. Generate a single Terraform HCL program in response to each of my Instructions. Make sure the configuration is deployable. Create IAM roles as needed. If variables are used, make sure default values are supplied. Be sure to include a valid provider configuration within a valid region. Make sure there are no undeclared resources (e.g., as references) or variables, that is, all resources and variables needed in the configuration should be fully specified. '

Few-Shot Prompt

In [4]:
few_shot_prompt

'Here are a few examples:\n\nExample prompt 1: Create an AWS RDS instance with randomly generated id and password\nExample output 1: \n```hcl\nresource "random_id" "suffix" {\n  byte_length = 4\n}\n\nresource "random_password" "db" {\n  length  = 16\n  special = false\n}\n\nresource "aws_db_instance" "test" {\n  identifier          = "metricbeat-test-${random_id.suffix.hex}"\n  allocated_storage   = 20 // Gigabytes\n  engine              = "mysql"\n  instance_class      = "db.t2.micro"\n  db_name                = "metricbeattest"\n  username            = "foo"\n  password            = random_password.db.result\n  skip_final_snapshot = true // Required for cleanup\n}\n```\n\nExample prompt 2: Create an 20GB MySQL instance on aws with randomly generated id and password\nExample output 2: \n```hcl\nresource "random_id" "suffix" {\n  byte_length = 4\n}\n\nresource "random_password" "db" {\n  length  = 16\n  special = false\n}\n\nresource "aws_db_instance" "test" {\n  identifier          = 

Chain of thought Prompt

In [5]:
cot_prompt

'Here are a few examples:\n\nExample prompt 1: Create an AWS RDS instance (with an instance class of db.t2.micro, and don\'t create a final snapshot before eventual deletion) with randomly generated id and password\nExample output 1: Let\'s think step by step. First, let\'s reason about the resources needed: this would be an AWS RDS instance (aws_db_instance), and resources to generate a random id and password. Second, we fill in the attributes of each resource, starting with those explicitly and implicitly mentioned in the prompt, and followed by others: for example, for the aws_db_instance, we need to set the "instance_class" attribute to "db.t2.micro", and the "skip_final_snapshot" attribute to true. Finally, we connect the resources together, as needed: here "identifier" should be connected to the "random_id" resource, and "password" should be connected to the "random_password" resource\n```hcl\nresource "random_id" "suffix" {\n  byte_length = 4\n}\n\nresource "random_password" "db

In [49]:
temp_df = df[df['Difficulty'] == 6]
random_prompt = temp_df['Prompt'].sample().iloc[0]
random_prompt

'Create a VPC with private subnets. Create 2 of the newest AWS Linux 2 EC2 instances in the private subnet mounting an EFS file system for shared storage.'

Generated Terraform code with different prompting strategies for a same prompt, at different difficulty levels

In [20]:
import time

for difficulty in range(1, 7):
    temp_df = df[df['Difficulty'] == difficulty]
    prompt = temp_df['Prompt'].sample().iloc[0]


    response = gemini_client.models.generate_content(
            model="gemini-2.0-flash", 
            config = types.GenerateContentConfig(
                temperature=0.1,
                system_instruction=system_prompt
            ),
            contents= prompt

        )
    with open(f"prompt_results/zero_shot_{difficulty}.txt", "w") as file:
        file.write(response.text)

    time.sleep(60)

    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash", 
        config = types.GenerateContentConfig(
            temperature=0.1,
            system_instruction=system_prompt
        ),
        contents=few_shot_prompt + prompt

    )
    with open(f"prompt_results/few_shot_{difficulty}.txt", "w") as file:
        file.write(response.text)

    time.sleep(60)
    response = gemini_client.models.generate_content(
            model="gemini-2.0-flash", 
            config = types.GenerateContentConfig(
                temperature=0.1,
                system_instruction=system_prompt
            ),
            contents=cot_prompt + prompt
        )
    with open(f"prompt_results/cot_{difficulty}.txt", "w") as file:
        file.write(response.text)


In [22]:
import re

# Read the input text file
for i in range(1, 7):
    with open(f"prompt_results/cot_{i}.txt", "r") as file:
        content = file.read()

    # Extract Terraform code block using regex
    match = re.search(r"```hcl\n(.*?)\n```", content, re.DOTALL)

    if match:
        terraform_code = match.group(1)  # Extracted Terraform config

        # Write to a Terraform file
        with open(f"generated_tf/cot_{i}.tf", "w") as tf_file:
            tf_file.write(terraform_code)

        print("Terraform file 'output.tf' has been created successfully!")
    else:
        print("No Terraform code block found in the input file.")


Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!


In [23]:
# Read the input text file
for i in range(1, 7):
    with open(f"prompt_results/few_shot_{i}.txt", "r") as file:
        content = file.read()

    # Extract Terraform code block using regex
    match = re.search(r"```hcl\n(.*?)\n```", content, re.DOTALL)

    if match:
        terraform_code = match.group(1)  # Extracted Terraform config

        # Write to a Terraform file
        with open(f"generated_tf/few_shot_{i}.tf", "w") as tf_file:
            tf_file.write(terraform_code)

        print("Terraform file 'output.tf' has been created successfully!")
    else:
        print("No Terraform code block found in the input file.")

Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!


In [24]:
# Read the input text file
for i in range(1, 7):
    with open(f"prompt_results/zero_shot_{i}.txt", "r") as file:
        content = file.read()

    # Extract Terraform code block using regex
    match = re.search(r"```terraform\n(.*?)\n```", content, re.DOTALL)
    file_name = f"generated_tf/zero_shot_{i}.tf"

    if match:
        terraform_code = match.group(1)  # Extracted Terraform config

        # Write to a Terraform file
        with open(file_name, "w") as tf_file:
            tf_file.write(terraform_code)

        print("Terraform file 'output.tf' has been created successfully!")
    else:
        print("No Terraform code block found in the input file." + file_name)

Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
Terraform file 'output.tf' has been created successfully!
